In [55]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import dbfread
from dbfread import DBF
import csv

import warnings; warnings.simplefilter('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
import sklearn

import mglearn
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [56]:
data = pd.read_csv('Energi_Viborg_Dandas_data.csv')

data[0:-1]

,ID,mslink,XKoordinat,YKoordinat,LedningID,Dobbeltled,EjerKompon,fra_kote,til_kote,Laengde,...,anlag_aar,TransportK,SystemKode,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri,DatoUdf
0,87527,87527,526809.08,6250849.80,87527,1.0,1.0,4.500000,3.539986,2.55,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
1,87528,87528,526794.78,6250869.11,87528,1.0,1.0,4.930000,4.500000,24.03,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
2,87530,87530,526772.22,6250871.70,87530,1.0,1.0,5.180000,4.930000,22.71,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
3,87531,87531,526761.65,6250892.87,87531,1.0,1.0,6.247029,5.180000,23.67,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
4,87539,87539,526729.74,6250869.41,87539,1.0,1.0,5.530000,6.344251,39.61,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24137,223236,223236,540562.07,6249035.25,223051,1.0,1.0,21.080000,19.130000,51.54,...,2018.0,1,0,0,1.0,0.0,2018,2018,NaN,2018.0
24138,223238,223238,540611.81,6249084.37,223053,1.0,1.0,20.140000,19.550000,48.71,...,2018.0,1,1,0,1.0,0.0,2018,2018,NaN,2018.0
24139,223239,223239,540612.39,6249083.67,223054,1.0,1.0,19.710000,19.120000,46.98,...,2018.0,1,0,0,1.0,0.0,2018,2018,NaN,2018.0
24140,223240,223240,540733.23,6249089.23,223055,1.0,1.0,21.720000,20.140000,121.52,...,2018.0,1,1,0,1.0,0.0,2018,2018,NaN,2018.0


In [57]:
print("Number of rows before removing NaNs: {}".format(data.shape[0]))


Number of rows before removing NaNs: 24143


In [58]:
#get data copied
datafull = data
datacopy=data

In [59]:
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows after removing NaNs: 1258


In [60]:
#drop columns not needed
columns_to_be_removed = ['ID','mslink','XKoordinat','YKoordinat','Dobbeltled','EjerKompon','TransportK','SystemKode','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')
data[0:-1]

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,87810,34.72,33.48,64.88,19.112207,300.0,1.0,1939.0,0,1.0,0.0,2010,2014,1997.0
118,88211,54.47,54.40,4.90,14.285714,160.0,4.0,2011.0,0,1.0,0.0,2013,2014,2012.0
297,89021,49.79,49.39,14.68,27.247956,191.0,4.0,1975.0,0,1.0,0.0,2010,2014,2016.0
298,89021,49.79,49.39,14.68,27.247956,191.0,4.0,1975.0,0,1.0,0.0,2016,2016,2016.0
299,89025,50.36,49.81,47.33,11.620537,191.0,1.0,1975.0,0,1.0,0.0,2010,2014,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23613,219615,45.20,44.63,41.16,13.848397,191.0,4.0,1970.0,0,1.0,0.0,2018,2018,2002.0
23696,220181,29.38,26.74,84.72,31.161473,335.0,1.0,1920.0,0,1.0,0.0,2018,2018,2018.0
23722,220393,19.35,12.99,219.36,28.993435,450.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0
23723,220396,26.32,19.35,225.27,30.940649,400.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0


In [61]:
#get data with only TVObsKode as 1,which means observed broken by TV and man
data_with_TVObsAndMan = data[data['TVObsKode'].isin([1])]
data_with_TVObsAndMan = data_with_TVObsAndMan.drop(data_with_TVObsAndMan[data_with_TVObsAndMan['DatoOprett']==data_with_TVObsAndMan['DatoSaneri']].index)
data_with_TVObsAndMan = data_with_TVObsAndMan.drop(data_with_TVObsAndMan[data_with_TVObsAndMan['DatoOpdate']==data_with_TVObsAndMan['DatoSaneri']].index)

data_with_TVObsAndMan[0:-1]


,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
811,91329,17.86,17.38,16.30,29.447853,482.0,4.0,1950.0,0,1.0,1.0,2010,2014,2000.0
817,91349,20.56,20.23,49.86,6.618532,479.0,4.0,1995.0,0,1.0,1.0,2010,2014,2003.0
893,92295,18.40,18.28,14.63,8.202324,185.0,4.0,1980.0,0,1.0,1.0,2010,2014,1994.0
1508,95695,42.94,41.65,90.77,14.211744,285.0,4.0,1992.0,0,1.0,1.0,2010,2014,2007.0
1527,95949,37.35,36.94,19.19,21.365294,310.0,1.0,1925.0,0,1.0,1.0,2010,2014,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20936,207536,35.81,35.68,44.37,2.929908,191.0,4.0,1954.0,0,1.0,1.0,2016,2016,2017.0
20940,207545,36.28,36.05,47.96,4.795663,191.0,4.0,1954.0,0,1.0,1.0,2016,2016,2017.0
21738,211010,48.30,47.53,44.00,17.500000,190.0,4.0,1964.0,0,1.0,1.0,2016,2016,2017.0
21881,211896,50.02,49.35,64.48,10.390819,241.0,4.0,1970.0,0,1.0,1.0,2016,2016,2017.0


In [62]:
print("Number of rows with both TVObs and manObs: {}".format(data_with_TVObsAndMan.shape[0]))

Number of rows with both TVObs and manObs: 68


In [63]:
# get the rest which means only observed broken by man
data_with_only_manObs = data[~data['LedningID'].isin(data_with_TVObsAndMan['LedningID'])]
data_with_only_manObs[0:-1]

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,87810,34.72,33.48,64.88,19.112207,300.0,1.0,1939.0,0,1.0,0.0,2010,2014,1997.0
118,88211,54.47,54.40,4.90,14.285714,160.0,4.0,2011.0,0,1.0,0.0,2013,2014,2012.0
297,89021,49.79,49.39,14.68,27.247956,191.0,4.0,1975.0,0,1.0,0.0,2010,2014,2016.0
298,89021,49.79,49.39,14.68,27.247956,191.0,4.0,1975.0,0,1.0,0.0,2016,2016,2016.0
299,89025,50.36,49.81,47.33,11.620537,191.0,1.0,1975.0,0,1.0,0.0,2010,2014,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23613,219615,45.20,44.63,41.16,13.848397,191.0,4.0,1970.0,0,1.0,0.0,2018,2018,2002.0
23696,220181,29.38,26.74,84.72,31.161473,335.0,1.0,1920.0,0,1.0,0.0,2018,2018,2018.0
23722,220393,19.35,12.99,219.36,28.993435,450.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0
23723,220396,26.32,19.35,225.27,30.940649,400.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0


In [64]:
print("Number of rows with only manObs: {}".format(data_with_only_manObs.shape[0]))

Number of rows with only manObs: 1187


In [65]:
#test if datafull copied all data in data
print("Number of rows in datafull: {}".format(datafull.shape[0]))

Number of rows in datafull: 24143


In [66]:
#get data only observed broken by TV
datafull=datafull.drop(columns_to_be_removed,axis='columns')
data_with_only_TVObs = datafull[datafull['TVObsKode'].isin([1])]
data_with_only_TVObs[0:-1]

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
42,87832,39.460000,39.16,91.75,3.269755,400.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
43,87834,39.710000,39.48,87.69,2.622876,300.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
64,87901,40.550000,40.08,52.11,9.019382,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
65,87903,40.380000,40.55,68.39,-2.485744,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
74,87974,26.516191,26.86,72.20,-4.761893,250.0,1.0,1965.0,10,1.0,1.0,2010,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23953,221994,9.030000,7.71,56.28,23.454158,350.0,1.0,1968.0,0,1.0,1.0,2018,2018,NaN
23955,222006,10.540000,9.04,66.90,22.421525,350.0,1.0,1968.0,0,1.0,1.0,2018,2018,NaN
23957,222012,11.940000,10.55,60.32,23.043767,350.0,1.0,1968.0,0,1.0,1.0,2018,2018,NaN
24062,222265,25.720000,22.82,44.37,65.359477,160.0,4.0,2017.0,0,1.0,1.0,2018,2018,NaN


In [67]:
print("Number of rows with only TVObs: {}".format(data_with_only_TVObs.shape[0]))

Number of rows with only TVObs: 935


In [68]:
list = [data_with_only_TVObs ,data_with_only_manObs]
data_one_broke = pd.concat(list)
data_one_broke[0:-1]

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
42,87832,39.460000,39.16,91.75,3.269755,400.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
43,87834,39.710000,39.48,87.69,2.622876,300.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
64,87901,40.550000,40.08,52.11,9.019382,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
65,87903,40.380000,40.55,68.39,-2.485744,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
74,87974,26.516191,26.86,72.20,-4.761893,250.0,1.0,1965.0,10,1.0,1.0,2010,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23613,219615,45.200000,44.63,41.16,13.848397,191.0,4.0,1970.0,0,1.0,0.0,2018,2018,2002.0
23696,220181,29.380000,26.74,84.72,31.161473,335.0,1.0,1920.0,0,1.0,0.0,2018,2018,2018.0
23722,220393,19.350000,12.99,219.36,28.993435,450.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0
23723,220396,26.320000,19.35,225.27,30.940649,400.0,4.0,1980.0,0,1.0,0.0,2018,2018,2004.0


In [69]:
print("Number of rows with only one broken: {}".format(data_one_broke.shape[0]))

Number of rows with only one broken: 2122


In [70]:
#all broken pipes
all_list = [data_one_broke,data_with_TVObsAndMan]
data_all_broken = pd.concat(all_list)
data_all_broken[0:-1]

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
42,87832,39.460000,39.16,91.75,3.269755,400.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
43,87834,39.710000,39.48,87.69,2.622876,300.0,1.0,1939.0,0,1.0,1.0,2010,2014,NaN
64,87901,40.550000,40.08,52.11,9.019382,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
65,87903,40.380000,40.55,68.39,-2.485744,250.0,1.0,1945.0,0,1.0,1.0,2010,2014,NaN
74,87974,26.516191,26.86,72.20,-4.761893,250.0,1.0,1965.0,10,1.0,1.0,2010,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20936,207536,35.810000,35.68,44.37,2.929908,191.0,4.0,1954.0,0,1.0,1.0,2016,2016,2017.0
20940,207545,36.280000,36.05,47.96,4.795663,191.0,4.0,1954.0,0,1.0,1.0,2016,2016,2017.0
21738,211010,48.300000,47.53,44.00,17.500000,190.0,4.0,1964.0,0,1.0,1.0,2016,2016,2017.0
21881,211896,50.020000,49.35,64.48,10.390819,241.0,4.0,1970.0,0,1.0,1.0,2016,2016,2017.0


In [71]:
print("Number of rows with only all broken: {}".format(data_all_broken.shape[0]))

Number of rows with only all broken: 2190


In [73]:
#get around 2000 rows with nan
datacopy = datacopy[datacopy['DatoSaneri'].isnull()]#man check as null
datacopy = datacopy.loc[datacopy['TVObsKode']==0]#tvchecked as null
datacopy[0:-1]

,ID,mslink,XKoordinat,YKoordinat,LedningID,Dobbeltled,EjerKompon,fra_kote,til_kote,Laengde,...,anlag_aar,TransportK,SystemKode,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri,DatoUdf
0,87527,87527,526809.08,6250849.80,87527,1.0,1.0,4.500000,3.539986,2.55,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
1,87528,87528,526794.78,6250869.11,87528,1.0,1.0,4.930000,4.500000,24.03,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
2,87530,87530,526772.22,6250871.70,87530,1.0,1.0,5.180000,4.930000,22.71,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
3,87531,87531,526761.65,6250892.87,87531,1.0,1.0,6.247029,5.180000,23.67,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
4,87539,87539,526729.74,6250869.41,87539,1.0,1.0,5.530000,6.344251,39.61,...,1996.0,1,1,0,1.0,0.0,2010,2014,NaN,1996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24137,223236,223236,540562.07,6249035.25,223051,1.0,1.0,21.080000,19.130000,51.54,...,2018.0,1,0,0,1.0,0.0,2018,2018,NaN,2018.0
24138,223238,223238,540611.81,6249084.37,223053,1.0,1.0,20.140000,19.550000,48.71,...,2018.0,1,1,0,1.0,0.0,2018,2018,NaN,2018.0
24139,223239,223239,540612.39,6249083.67,223054,1.0,1.0,19.710000,19.120000,46.98,...,2018.0,1,0,0,1.0,0.0,2018,2018,NaN,2018.0
24140,223240,223240,540733.23,6249089.23,223055,1.0,1.0,21.720000,20.140000,121.52,...,2018.0,1,1,0,1.0,0.0,2018,2018,NaN,2018.0


In [74]:
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy = datacopy.sample(n=2100)

In [75]:
datacopy[0:-1]
#datacopy now is random 2100 data rows selected for data group "no broken".

,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
15995,183973,2.24,2.64,4.61,-86.767896,450.0,4.0,2010.0,0,1.0,0.0,2011,2014,NaN
1985,98936,12.04,11.62,25.62,16.393443,400.0,1.0,1938.0,0,1.0,0.0,2010,2014,NaN
11890,151526,16.47,16.43,9.97,4.012036,300.0,1.0,1946.0,10,1.0,0.0,2010,2014,NaN
20291,204867,38.00,36.51,104.07,14.317286,315.0,4.0,2015.0,0,1.0,0.0,2015,2015,NaN
1631,96732,41.30,40.82,46.50,10.322581,250.0,1.0,1942.0,0,1.0,0.0,2010,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9183,136522,20.03,19.97,5.86,10.238908,250.0,1.0,1965.0,0,1.0,0.0,2010,2014,NaN
15993,183970,4.38,4.79,8.24,-49.757282,200.0,4.0,2010.0,0,1.0,0.0,2011,2014,NaN
8819,134350,38.86,36.99,16.52,113.196126,225.0,4.0,2000.0,0,1.0,0.0,2010,2014,NaN
8817,134344,36.53,35.16,39.62,34.578496,360.0,4.0,2000.0,0,1.0,0.0,2010,2014,NaN


In [76]:
#add a column for "Time of broken"
drop_cols = ['DatoOprett','DatoOpdate','DatoSaneri']

#one_broken
data_one_broke = data_one_broke.drop(drop_cols,axis = 'columns')
data_one_broke.insert(data_one_broke.shape[1],'Times_Broken',1)
#twice_broken
data_with_TVObsAndMan = data_with_TVObsAndMan.drop(drop_cols,axis = 'columns')
data_with_TVObsAndMan.insert(data_with_TVObsAndMan.shape[1],'Times_Broken',2)
#no broken
datacopy = datacopy.drop(drop_cols,axis = 'columns')
datacopy.insert(datacopy.shape[1],'Times_Broken',0)

#merge into one table
list_zeroonetwo=[datacopy,data_one_broke,data_with_TVObsAndMan]
final_data_with_all_times_broken=pd.concat(list_zeroonetwo)

print("Number of rows with three types of broken: {}".format(final_data_with_all_times_broken.shape[0]))
final_data_with_all_times_broken[0:-1]

Number of rows with three types of broken: 4190


,LedningID,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,Funktionsk,KategoriAf,TVObsKode,Times_Broken
15995,183973,2.24,2.64,4.61,-86.767896,450.0,4.0,2010.0,0,1.0,0.0,0
1985,98936,12.04,11.62,25.62,16.393443,400.0,1.0,1938.0,0,1.0,0.0,0
11890,151526,16.47,16.43,9.97,4.012036,300.0,1.0,1946.0,10,1.0,0.0,0
20291,204867,38.00,36.51,104.07,14.317286,315.0,4.0,2015.0,0,1.0,0.0,0
1631,96732,41.30,40.82,46.50,10.322581,250.0,1.0,1942.0,0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20936,207536,35.81,35.68,44.37,2.929908,191.0,4.0,1954.0,0,1.0,1.0,2
20940,207545,36.28,36.05,47.96,4.795663,191.0,4.0,1954.0,0,1.0,1.0,2
21738,211010,48.30,47.53,44.00,17.500000,190.0,4.0,1964.0,0,1.0,1.0,2
21881,211896,50.02,49.35,64.48,10.390819,241.0,4.0,1970.0,0,1.0,1.0,2
